# new_create_db.ipynb

## Verification of detected objects

An **is_valid** column was added to the boxes and polygons tables as a mechanism for tagging false positives generated by the object detectors.
The idea is to have a human view object detector results to decide on which ones are valid and which should be regarded as false positives. 
The **is_valid** field can have one of 3 values:

value | meaning
--- | ---
U | Unknown - the object has not been verified by a human (this is the default value)
F | False - a human has examined the object and has decided that it is a false positive
T | True - a human has examined the object and has decided that it is a valid detection

Here is how the field is specified in SQL when the table is created:

```
is_valid TEXT CHECK (is_valid IN ('T','F', 'U')) NOT NULL DEFAULT 'U',
```

Note that a whitelist and/or a blacklist needs to be maintained to recover verified data whenever the database is rebuilt.
For blacklisted records, **is_valid** should be give an value of 'F'.
For whitelisted records, **is_valid** should be give a value of 'T'.
For unverified records, **is_valid** should remain as 'U' (the default value).

Here is an example of filtering out false positives using SQL. Note that records with **is_valid** values of 'T' or 'U' will be included in the results.

```
SELECT * FROM polygons WHERE is_valid != 'F';
```

In [1]:
# parameters for papermill
DBPATH = '700_990.db'
DATADIR = '../rawdata'
MIN_BOX_SCORE = 0.700
MIN_POLYGON_SCORE = 0.990
WHITELISTED_POLYGON_IDS = '87,88,263'
WHITELISTED_D5 = '191,677'

In [2]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 207 µs (started: 2023-12-06 05:01:22 +08:00)


In [3]:
# import geopandas as gpd
# from shapely.geometry import Point, LineString
import os
import glob
import spatialite
import pandas as pd
# import xml.etree.ElementTree as ET
import pickle
import numpy as np
import re

time: 252 ms (started: 2023-12-06 05:01:22 +08:00)


In [4]:
# def delete_polygons_not_whitelisted():
#     sql = f'DELETE FROM polygons WHERE id NOT IN ({WHITELISTED_POLYGON_IDS})'
#     cursor.execute(sql)
#     conn.commit()     

time: 167 µs (started: 2023-12-06 05:01:22 +08:00)


In [5]:
def execute(command):
    """
    Executes a command line.
    An error is raised if the command fails, stopping the notebook.
    """
    if os.system(command) != 0:
        raise Exception(f'Command failed: {command}')

time: 6.1 ms (started: 2023-12-06 05:01:22 +08:00)


In [6]:
def populate_images_table():
    gps_csv_path = f'{DATADIR}/gps-data.csv'
    df = pd.read_csv(gps_csv_path)
    for i, r in df.iterrows():
        imagefile = r.imagefile
        lat = r.latitude_adjusted
        lon = r.longitude_adjusted
        geometry = f'GeomFromText("POINT({lon} {lat})", 4326)'       
        sql = f'INSERT INTO images (id, imagefile, geometry) VALUES (Null, "{imagefile}", {geometry});'
        cursor.execute(sql)
    conn.commit()

time: 2.08 ms (started: 2023-12-06 05:01:22 +08:00)


In [7]:
def process_pickle_data(pickle_file_path):
    results = {}
    
    original_image_path = pickle_file_path.replace('.pkl', '.jpg')
    results['original_image_path'] = original_image_path
        
    with open(pickle_file_path, 'rb') as f:
        data = pickle.load(f)

    width = data.get('width') 
    results['width'] = width
    
    height = data.get('height')
    results['height'] = height
    
    num_detections = data.get('num_detections')
    
    if not num_detections:
        results['num_detections'] = 0
    else:
        num_detections = int(num_detections[0])
        results['num_detections'] = num_detections

        scores = data.get('scores')[0]
        scores = scores[:num_detections]
        results['scores'] = scores

        boxes = data.get('boxes')[0]
        boxes = boxes[:num_detections]
        results['boxes'] = boxes

        classes = data.get('classes')[0]
        classes = classes.astype(int)
        classes = classes[:num_detections]
        results['classes'] = classes

    polygons_scores = data.get('polygons_scores')
    if not polygons_scores:
        results['num_polygons'] = 0
    else:
        num_polygons = len(polygons_scores)
        results['num_polygons'] = num_polygons
 
        results['polygons_scores'] = polygons_scores
       
        polygons_result = data.get('polygons_result')
        polygons = polygons_result.get('cut')
        polygons = polygons[:num_polygons]    
        results['polygons'] = polygons          
    return results
    
# process_pickle_data('../rawdata/IMG_20231007_110848.pkl')
# process_pickle_data(pickle_files[1])

time: 2.4 ms (started: 2023-12-06 05:01:22 +08:00)


In [8]:
def process_detected_objects():
    pickle_files = sorted(glob.glob('../rawdata/*.pkl')) 
    boxes_list = []
    polygons_list = []
    for pickle_file_path in pickle_files:
        shapes = process_pickle_data(pickle_file_path)
        num_detections = shapes['num_detections']
        if num_detections > 0:
            width = shapes['width']
            height = shapes['height']       
            original_image_path = shapes['original_image_path']
            scores = list(shapes['scores'])
            boxes = list(shapes['boxes'])
            damage_levels = list(shapes['classes'])         
            
            for i, box in enumerate(boxes):
                boxes_list.append({
                    'b_original_image_path': original_image_path,
                    'b_score': scores[i], 
                    'b_damage_level': damage_levels[i],
                    'b_ulx': int(round(box[1]*width)),
                    'b_uly': int(round(box[0]*height)),
                    'b_lrx': int(round(box[3]*width)),
                    'b_lry': int(round(box[2]*height))
                })
            
            num_polygons = shapes['num_polygons']
            if num_polygons > 0:
                polygons = shapes['polygons']
                polygons_scores = shapes['polygons_scores']
                for i in range(num_polygons):
                    points_flt = polygons[i]
                    points_int = [int(i) for i in points_flt]           
                    points = np.array(points_int, np.int32)
                    points_2d = np.reshape(points, (-1, 2))

                    # calculate bounding box for polygon

                    s = list(points_2d.flatten())
                    xlist = s[0::2]
                    ylist = s[1::2]
                    ulx = min(xlist)
                    uly = min(ylist)
                    lrx = max(xlist)
                    lry = max(ylist)

                    # polygon is stored as a string to facilitate storage in SQLite. 
                    # Use eval() to convert back to a list of integers [x0, y0, x1, y1, ..., x0, y0].
                    
                    polygons_list.append({
                        'p_original_image_path': original_image_path, 
                        'p_score': polygons_scores[i], 
                        'p_polygon': str(s),  
                        'p_ulx': ulx,
                        'p_uly': uly,
                        'p_lrx': lrx,
                        'p_lry': lry
                    })
                    
        df_boxes = pd.DataFrame(boxes_list)
        df_polygons = pd.DataFrame(polygons_list)
    return df_boxes, df_polygons

time: 3.24 ms (started: 2023-12-06 05:01:22 +08:00)


In [9]:
def populate_boxes_table():
    for i, r in df_boxes.iterrows():
        if r.b_score < MIN_BOX_SCORE:
            continue
        wkt = f'POLYGON (({r.b_ulx} {r.b_uly}, {r.b_ulx} {r.b_lry}, {r.b_lrx} {r.b_lry}, {r.b_lrx} {r.b_uly}, {r.b_ulx} {r.b_uly}))'
        geometry = f'GeomFromText("{wkt}", -1)' 

        # Get images_id
        imagefile = os.path.basename(r.b_original_image_path)
        sql = f'SELECT id FROM images WHERE imagefile="{imagefile}";'
        cursor.execute(sql)
        images_id = cursor.fetchone()[0] 

        sql = f'''INSERT INTO boxes (id, score, damage_level, geometry, images_id) 
        VALUES (Null, {r.b_score}, {r.b_damage_level}, {geometry}, {images_id});'''
        cursor.execute(sql) 
        
    print('------processing whitelist for damage_level=5')
    sql = f"UPDATE boxes SET is_valid = 'T' WHERE id IN ({WHITELISTED_D5}) AND damage_level=5;"
    cursor.execute(sql)
    sql = f"UPDATE boxes SET is_valid = 'F' WHERE id NOT IN ({WHITELISTED_D5}) AND damage_level=5;"
    cursor.execute(sql)

    conn.commit()    

time: 1.8 ms (started: 2023-12-06 05:01:22 +08:00)


In [10]:
def populate_polygons_table():
    for i, r in df_polygons.iterrows():
        if r.p_score < MIN_POLYGON_SCORE:
            continue
        
        # reformat polygon to WKT
        myString = r.p_polygon
        myString = re.sub(',([^,]*,?)', r'\1', myString) # remove every second string
        myString = myString.replace('[', 'POLYGON ((').replace(']', '))')
        wkt = myString
        geometry = f'GeomFromText("{wkt}", -1)' 

        # Get images_id
        imagefile = os.path.basename(r.p_original_image_path)
        sql = f'SELECT id FROM images WHERE imagefile="{imagefile}";'
        cursor.execute(sql)
        images_id = cursor.fetchone()[0] 

        sql = f'''
        INSERT INTO polygons (id, score, geometry, images_id) 
          VALUES (Null, {r.p_score}, {geometry}, {images_id});
        '''
        cursor.execute(sql)
        
    print('------processing whitelist for polygons')
    sql = f"UPDATE polygons SET is_valid = 'T' WHERE id IN ({WHITELISTED_POLYGON_IDS});"
    cursor.execute(sql)
    sql = f"UPDATE polygons SET is_valid = 'F' WHERE id NOT IN ({WHITELISTED_POLYGON_IDS});"
    cursor.execute(sql)
    
    conn.commit() 
    
# populate_polygons_table()

time: 2.31 ms (started: 2023-12-06 05:01:22 +08:00)


In [11]:
def populate_intersections_table():
    sql = '''
    SELECT boxes.images_id, boxes.id AS boxes_id, boxes.geometry AS bgeom, polygons.id AS polygons_id, polygons.geometry AS pgeom 
    FROM boxes, polygons 
    WHERE (boxes.images_id=polygons.images_id)
      AND boxes.is_valid != 'F'
      AND polygons.is_valid != 'F'
      AND ST_Intersects(bgeom, pgeom);
    '''
    df_interceptions = pd.read_sql(sql, conn)

    for i, r in df_interceptions.iterrows():   
        sql = f'INSERT INTO intersections (id, boxes_id, polygons_id) VALUES (Null, {r.boxes_id}, {r.polygons_id})'
        cursor.execute(sql) 
    conn.commit() 
    
    # I found a small number of records in the polygons table where geometry is null. 
    # The following lines fix this problem temporarily.
    
    sql = 'DELETE FROM polygons WHERE geometry IS NULL;'
    cursor.execute(sql)
    conn.commit()

time: 3.8 ms (started: 2023-12-06 05:01:22 +08:00)


In [12]:
def create_tables():
    
    sql = 'SELECT InitSpatialMetadata(1);'
    cursor.execute(sql)
    
    sql = """
    CREATE TABLE images (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        imagefile TEXT NOT NULL UNIQUE,
        geometry);
    """
    cursor.execute(sql)
    sql = "SELECT RecoverGeometryColumn('images', 'geometry', 4326, 'POINT', 'XY');"
    cursor.execute(sql)
    
    sql = """
    CREATE TABLE boxes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,   
        score REAL,
        damage_level INTEGER,
        images_id INTEGER,
        is_valid TEXT CHECK (is_valid IN ('T','F', 'U')) NOT NULL DEFAULT 'U',
        geometry,
        FOREIGN KEY(images_id) REFERENCES images(id));
        """
    cursor.execute(sql)
    sql = "SELECT RecoverGeometryColumn('boxes', 'geometry', -1, 'POLYGON', 'XY');"
    cursor.execute(sql)
    
    sql = """
    CREATE TABLE polygons (
        id INTEGER PRIMARY KEY AUTOINCREMENT,  
        score REAL,
        is_valid TEXT CHECK (is_valid IN ('T','F', 'U')) NOT NULL DEFAULT 'U',
        geometry,
        images_id INTEGER,
        FOREIGN KEY(images_id) REFERENCES images(id));
    """
    cursor.execute(sql)
    sql = "SELECT RecoverGeometryColumn('polygons', 'geometry', -1, 'POLYGON', 'XY');"
    cursor.execute(sql)

    sql = """
    CREATE TABLE intersections (
        id INTEGER PRIMARY KEY AUTOINCREMENT,  
        boxes_id INTEGER,
        polygons_id INTEGER,
        FOREIGN KEY(boxes_id) REFERENCES boxes(id),
        FOREIGN KEY(polygons_id) REFERENCES polygons(id));
    """
    cursor.execute(sql)
    
    sql = """
    CREATE TABLE stats (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL UNIQUE,
        val TEXT );
    """
    cursor.execute(sql)
    
    conn.commit()

time: 3.95 ms (started: 2023-12-06 05:01:22 +08:00)


In [13]:
def adjust_damage_index():
    """Adjust the damage index associated with each detected coconut palm using the following rules.
    
    1. If the current index is 1 (no damage), and one or more polygons (v-shaped cuts) intersect the area within the palm's bounding box, 
    then the index is increased to 2 (light damage). In other words, a palm with no damage cannot have v-shaped cuts.
    
    2. If the index is currently 2 or 3 (light damage or medium damage), and there are no v-shaped cuts intersecting the palm's bounding box, 
    then the damage index is reduced to 1 (no damage). In other words, index 2 or 3 requires presence of one or more v-shaped cuts. Note that
    v-shaped cuts may not be apparent on palms with index 4 (severe damage) and 5 (dead or moribund).
    """
    
    sql = """
    SELECT boxes.id, damage_level, count(polygons_id) as vcuts
    FROM boxes
    LEFT JOIN intersections
    ON boxes.id = intersections.boxes_id
    GROUP BY boxes.id
    ORDER BY vcuts DESC;
    """
    df = pd.read_sql(sql, conn)       
    n_increased = 0
    n_decreased = 0
    for i, r in df.iterrows():
        if r.damage_level == 1 and r.vcuts > 0:
            sql = f'UPDATE boxes SET damage_level = 2 WHERE id = {r.id};'
            cursor.execute(sql)
            n_increased += 1
        if r.damage_level in (2,3) and r.vcuts == 0:
            sql = f'UPDATE boxes SET damage_level = 1 WHERE id = {r.id};'
            cursor.execute(sql)
            n_decreased +=1
    conn.commit() 
    print(f'------damage_level was increased from 1 to 2 for {n_increased} palms')
    print(f'------damage_level was decreased from 2 or 3 to 1 for {n_decreased} palms')

# conn = spatialite.connect(DBPATH)
# cursor = conn.cursor()
# adjust_damage_index()
# conn.close()    

time: 2.63 ms (started: 2023-12-06 05:01:22 +08:00)


In [14]:
def populate_stats_table():    
    cursor.execute("INSERT INTO stats (name, val) SELECT 'images_count', COUNT(*) FROM images;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'trees_count', COUNT(*) FROM boxes;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'valid_trees_count', COUNT(*) FROM boxes WHERE is_valid != 'F';")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'trees_damage_1', COUNT(*) FROM boxes WHERE damage_level=1;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'trees_damage_2', COUNT(*) FROM boxes WHERE damage_level=2;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'trees_damage_3', COUNT(*) FROM boxes WHERE damage_level=3;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'trees_damage_4', COUNT(*) FROM boxes WHERE damage_level=4;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'trees_damage_5', COUNT(*) FROM boxes WHERE damage_level=5;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'trees_damaged', COUNT(*) FROM boxes WHERE damage_level>1;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'vcuts_count', COUNT(*) FROM polygons;")
    cursor.execute("INSERT INTO stats (name, val) SELECT 'valid_vcuts_count', COUNT(*) FROM polygons WHERE is_valid != 'F';")
    conn.commit()

time: 2.91 ms (started: 2023-12-06 05:01:22 +08:00)


In [15]:
# MAIN

# Delete the database so we can start from scratch

if os.path.exists(DBPATH):
    os.remove(DBPATH)

print('---creating database')
# execute(f'spatialite {DBPATH} < create_tables3.sql')
conn = spatialite.connect(DBPATH)
cursor = conn.cursor()

print('---creating tables')
create_tables()

print('---populating images table')
populate_images_table()

print('---getting data from pickle files')
df_boxes, df_polygons = process_detected_objects()

print('---populating boxes table')
populate_boxes_table()

print('---populating polygons table')
populate_polygons_table()

print('---populating intersections table')
populate_intersections_table()

print('---adjusting_damage_index')
adjust_damage_index()

print('---populating stats table')
populate_stats_table()

conn.close()

print('FINISHED')

---creating database
---creating tables
---populating images table
---getting data from pickle files
---populating boxes table
------processing whitelist for damage_level=5
---populating polygons table
------processing whitelist for polygons
---populating intersections table
---adjusting_damage_index
------damage_level was increased from 1 to 2 for 0 palms
------damage_level was decreased from 2 or 3 to 1 for 2806 palms
---populating stats table
FINISHED
time: 58.8 s (started: 2023-12-06 05:01:22 +08:00)
